In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !pip install -q -U bitsandbytes # 모델을 경량화해서 로드
# !pip install transformers
# !pip install -q -U git+https://github.com/huggingface/peft.git # LoRA 튜닝을 하게 해주는 라이브러리
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
# !pip install -q datasets
# !pip install trl
# !pip install accelerate

In [1]:
from transformers import AutoTokenizer, LlamaForCausalLM, TrainingArguments, TrainerCallback
from datasets import load_dataset, load_from_disk
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
import peft
from trl import SFTTrainer
import torch
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")

In [2]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained('beomi/llama-2-ko-7b', quantization_config=bnb_config, device_map={'': 0})
# fine-tuning으로 사용할 foundation model의 tokenizer 사용
tokenizer = AutoTokenizer.from_pretrained('beomi/llama-2-ko-7b')
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

bin C:\Users\user\anaconda3\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


In [3]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(46336, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )


In [4]:
tokenizer

LlamaTokenizerFast(name_or_path='beomi/llama-2-ko-7b', vocab_size=46336, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	46331: AddedToken("<|sep|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	46332: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	46333: AddedToken("<|acc|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	46334: A

In [5]:
# train = pd.read_csv('/content/drive/MyDrive/데이콘/train_preprocess/train_preprocess_data_aug.csv')
# train = pd.read_csv('train_preprocess.csv')
train = pd.read_csv('../train_preprocess_data_aug.csv')
train

,질문,답변
0,면진장치가 뭐야?,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...
1,면진장치가 뭐야?,"면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을..."
2,면진장치가 뭐야?,면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여...
3,면진장치가 뭐야?,면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 ...
4,면진장치가 뭐야?,면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동...
...,...,...
6997,현관문 주위 결로의 대책은 뭐야? 또한 현관문 주위 결로의 원인은 뭐야??,현관문 주위 결로의 대책은 규격제품을 사용하고 방화문 현관문 내부 종이 허니컴 사이...
6998,"부엌 및 욕실의 결로의 원인은 뭐야? 또한 AD, PD에 면한 벽체 결로에 대한 대...",부엌이나 욕실의 경우 외벽이나 비 난방공간에 접한 면이 벽체의 일부인 부엌이나 욕실...
6999,발코니에 결로가 발생하는 원인은 뭐야? 또한 벽장 부위 결로의 원인이 뭐야??,발코니에 결로가 발생하는 원인은 발코니에 단열 및 방습조치가 되지 않은 상태에서 실...
7000,부엌 및 욕실의 결로의 대책은 뭐야? 또한 외벽 모서리 부위에 결로가 발생하는 원인...,부엌 및 욕실의 결로에 대한 대책은 환기구 설치입니다. 부엌 및 욕실의 결로는 주로...


In [6]:
dataset = []
for question, answer in tqdm(zip(train['질문'], train['답변']), total = len(train)):
    dataset.append(f'<s>### Instruction: \n{question} \n\n### Response: \n{answer}</s>')
dataset = Dataset.from_dict({"text": dataset})
dataset = dataset.shuffle()
dataset

  0%|          | 0/7002 [00:00<?, ?it/s]

Dataset({
    features: ['text'],
    num_rows: 7002
})

In [7]:
dataset['text'][0]

'<s>### Instruction: \n벽에 구멍을 막는 방법은? \n\n### Response: \n벽에 구멍을 메우기 위해서는 먼저 구멍 주변을 깨끗하게 솔로 닦아내고, 모래지거나 부서진 부분을 제거해야 합니다. 그 후에 구멍을 메우기 위해 합성 먼지 또는 석고로 메우고 필요하다면 패팅을 적용하면 됩니다. 이렇게 함으로써 벽의 평탄성을 회복시키고 페인트를 바를 수 있는 상태로 만들 수 있습니다.</s>'

In [8]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [9]:
lora_config_dict = {
    'lora_r' : 32,
    'lora_alpha' : 16,
    'lora_dropout' : 0.1,
    'lora_bias' : "none",
    'lora_task_type' : "CAUSAL_LM",
#     'target_modules': ["q_proj", "up_proj", "o_proj", "k_proj", "down_proj", "gate_proj", "v_proj"]
}

peft_config = peft.LoraConfig(
            lora_alpha=lora_config_dict['lora_alpha'],
            lora_dropout=lora_config_dict['lora_dropout'],
            r=lora_config_dict['lora_r'],
            bias=lora_config_dict['lora_bias'],
            task_type=lora_config_dict['lora_task_type'],
#             target_modules = lora_config_dict['target_modules']
        )

In [10]:
# output_dir = '/content/drive/MyDrive/데이콘/train_preprocess/llama2_lora_preprocess_data_aug'
output_dir = 'llama2_lora_preprocess_data_aug'

In [11]:
train_param = {
    'num_train_epochs': 5,
    'per_device_train_batch_size': 4,
    'per_device_eval_batch_size': 4,
    'gradient_accumulation_steps': 1,
    'optim': "adamw_torch",
    'save_steps': 1000,
    'logging_steps': 10,
    'learning_rate': 0.0002,
    'weight_decay':  0.01,
    'max_grad_norm': 1,
    'warmup_ratio': 0.06,
    'group_by_length' : False,
    'lr_scheduler_type': 'cosine'
}

training_args = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=train_param['num_train_epochs'],
            per_device_train_batch_size=train_param['per_device_train_batch_size'],
            per_device_eval_batch_size=train_param['per_device_eval_batch_size'],
            gradient_accumulation_steps=train_param['gradient_accumulation_steps'],
            optim=train_param['optim'],
            save_steps=train_param['save_steps'],
            logging_steps=train_param['logging_steps'],
            learning_rate=train_param['learning_rate'],
            weight_decay=train_param['weight_decay'],
            # fp16=kwargs['fp16'],
            # bf16=kwargs['bf16'],
            max_grad_norm=train_param['max_grad_norm'],
            # max_steps=kwargs['max_steps'],
            warmup_ratio=train_param['warmup_ratio'],
            group_by_length=train_param['group_by_length'],
            lr_scheduler_type=train_param['lr_scheduler_type'],
#             report_to="tensorboard"
        )

In [12]:
trainer = SFTTrainer(
            model=model,
            train_dataset=dataset,
            peft_config=peft_config,
            dataset_text_field="text",
            max_seq_length=None,
            tokenizer=tokenizer,
            args=training_args,
            packing=False,
            # neftune_noise_alpha=5,
            # callbacks=[AbortCallback()]
            )

Map:   0%|          | 0/7002 [00:00<?, ? examples/s]

In [13]:
model.config.use_cache = False
# trainer.train(resume_from_checkpoint=False)
trainer.train(resume_from_checkpoint=True)

Step,Training Loss
7010,0.534900
7020,0.514700
7030,0.656400
7040,0.541100
7050,0.505700
7060,0.543100
7070,0.567500
7080,0.532500
7090,0.534700
7100,0.504000


TrainOutput(global_step=8755, training_loss=0.10357600257847392, metrics={'train_runtime': 8787.045, 'train_samples_per_second': 3.984, 'train_steps_per_second': 0.996, 'total_flos': 2.3611498702022246e+17, 'train_loss': 0.10357600257847392, 'epoch': 5.0})

In [14]:
trainer.model.save_pretrained(output_dir)